In [13]:
import os
import shutil
import math
import datetime
import sys
import glob
import urllib
import tarfile
import urllib.request
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import re

import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np

import tensorflow as tf


In [2]:
print("Tensorflow version: ", tf.__version__)

Tensorflow version:  2.3.0


In [3]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 43399, done.
remote: Total 43399 (delta 0), reused 0 (delta 0), pack-reused 43399
Receiving objects: 100% (43399/43399), 550.10 MiB | 15.48 MiB/s, done.
Resolving deltas: 100% (29527/29527), done.


In [4]:
os.chdir('models/research/')
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
os.chdir('../..')

Processing /content/models/research
     |████████████████████████████████| 8.3MB 6.6MB/s 
     |████████████████████████████████| 358kB 37.5MB/s 
     |████████████████████████████████| 849kB 39.7MB/s 
     |████████████████████████████████| 829kB 43.7MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 1.4MB 40.4MB/s 
     |████████████████████████████████| 63.8MB 46kB/s 
     |████████████████████████████████| 153kB 41.1MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 1.1MB 38.2MB/s 
     |████████████████████████████████| 36.6MB 88kB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 174kB 42.1MB/s 
     |████████████████████████████████| 112kB 36.8MB/s 
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1568035 sha256=9e1a073753af193aec824489c17

In [5]:
os.chdir('models/research/')
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py
os.chdir('../..')

2020-09-04 14:56:33.117829: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-09-04 14:56:35.919946: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-04 14:56:35.975980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-04 14:56:35.976841: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-09-04 14:56:35.976891: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfull

In [6]:
!python data/xml_to_csv.py

python3: can't open file 'data/xml_to_csv.py': [Errno 2] No such file or directory


In [110]:
tf_records = 'data/tfrecords'
if not os.path.exists(tf_records):
  os.mkdir(tf_records)

!python data/create_tf_records.py --image-dir data/train  --labels training/train_labels --output_path training/tfrecords/ --labels-map training/label_map.pbtxt

2020-09-04 21:16:11.202094: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: data/train_labels.record
Traceback (most recent call last):
  File "data/create_tf_records.py", line 87, in <module>
    tf_example = create_tf_example(group, path)
  File "data/create_tf_records.py", line 44, in create_tf_example
    encoded_jpg = fid.read()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/lib/io/file_io.py", line 116, in read
    self._preread_check()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/lib/io/file_io.py", line 79, in _preread_check
    self.__name, 1024 * 512)
tensorflow.python.framework.errors_impl.NotFoundError: data/train/gettyimages-111487207-2048x2048.jpg; No such file or directory


In [28]:
MODELS_CONFIG = {
    'Mask_R-CNN-inception': {
        'model_name': 'mask_rcnn_inception_resnet_v2_1024x1024_coco17',
    },
}
selected_model = 'Mask_R-CNN-inception'

In [79]:
header = {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:78.0) Gecko/20100101 Firefox/78.0'}
MODEL = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8'

# Add the zip extension to the model
MODEL_FILE = MODEL + '.tar.gz'

# Define the base URL
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# If model directory is present then remove if for the new model
model_directory = "pretrained_model"
if os.path.exists(model_directory):
    shutil.rmtree(model_directory )

# Download the pretrained Model
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

# Here we are extracting the downloaded file
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# Removing the downloaded zip file.
os.remove(MODEL_FILE)

# Rename model directory to pretrained_model
os.rename(MODEL, model_directory)

# Remove the checkpoint file so the model can be trained
os.remove(model_directory + '/checkpoint')

print('Model Downloaded')

HTTPError: ignored

In [111]:
directory = 'data/pretrained_model/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/'
filename = directory + '/pipeline.config'

# Open the configutation file and read the whole file
with open(filename) as f:
    s = f.read()
    
# Now find and subsitute the source paths with the destinations paths.    
with open(filename, 'w') as f:
    s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', directory + '/model.ckpt', s)
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record', 'data/train_labels.record', s)
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record', 'data/test_labels.record', s)
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', 'data/label_map.pbtxt', s)
    
    # I have 3 classes (Sarfaraz Ahmed, MS Dhoni, Adam Gilchrist)
    s = re.sub('num_classes: 90', 'num_classes: 3', s)

    
    # Doing a little correction to avoid an error in training.
    s = re.sub('step: 0', 'step: 1', s)
            
    # I'm also changing the default batch_size of 1 to be 10 for this example
    s = re.sub('batch_size: 1', 'batch_size: 5', s)
    f.write(s)

In [117]:
!python models/research/object_detection/model_main.py  --pipeline_config_path="data/pretrained_model/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config" \
--model_dir="data/pretrained_model"  --num_train_steps=20000

2020-09-04 21:23:47.224442: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "models/research/object_detection/model_main.py", line 108, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "models/research/object_detection/model_main.py", line 70, in main
    FLAGS.sample_1_of_n_eval_on_train_examples))
  File "/usr/local/lib/python3.6/dist-packages/object_detection/model_lib.py", line 759, in create_estimator_and_inputs
    pipeline_config_path, config_override=config_override)
  File "/usr/local/lib/python3.6/dis